## Basic neural and network

![](1.png)

![](2.png)

In [33]:
import tensorflow as tf
x1= tf.constant(5)
x2=tf.constant(6)

result = tf.multiply(x1,x2)

print (result) # no computation happens here, we just defined a model, output is a tensor in computational block


with tf.Session():
    output = sess.run(result)
    print (output)

print (output)    


Tensor("Mul_6:0", shape=(), dtype=int32)
30
30


Let's build a simple DNN using tensorflow with MNIST data set.
MNSIT data set contains 60000 thousand 28*28 pixel B&W images of handwritten digitis

Objective is to predict digit with a new image.

Each of feature is a pixel value (i.e., 0 or 1) using these features our neural network is  going to model if the number if 7 or 11 or whatever

- We will send input data > weight > hidden layer > activation function > weights > hidden layer 2 > activation function > weights > output layer

- This is feedforward network
- We will compare the output with expected output using cost/loss function (ex: cross entropy)
- then we will use optimizer - this make an attempt to minimze the cost/loss (ex: AdamOptimizer, SGD, AdaGrad ... )
- What optimizer does is - it goes backwards, manipulates weights - this is backpropagation


- 1 Feed forward + 1 backprop = 1 Epoch
- After every epoch - the cost function will come down.





In [34]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data",one_hot=True)
# one hot is useful for multiclass classification - ex: digit 1 can be represented as 
# 010000000 and 3 can be represented as 0001000000
# 0 = [1,0,0,0,0,0,0,0,0,0]

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [44]:
# 3 hidden layers
n_nodes_h1=100 
n_nodes_h2=100
n_nodes_h3=100

In [45]:
# number of classes = 10
n_classes = 10 # outputs - 0 to 9

# batch size is number of images we will feed at one time 
# - we cannot feed 60000 images at one time - we can but depends on your RAM size
batch_size = 100 

# 28 * 28 = 784 - we have flattened our input. Height = None, Width = 784
x=tf.placeholder('float') 
y = tf.placeholder('float')

def model(data):
    # (input data * weights + biases)
    # Why bias? Imagine if any of the input data is zero?
    hidden_1_layer={'weights':tf.Variable(tf.random_normal([784,n_nodes_h1])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_h1]))}
    hidden_2_layer={'weights':tf.Variable(tf.random_normal([n_nodes_h1,n_nodes_h2])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_h2]))}
    hidden_3_layer={'weights':tf.Variable(tf.random_normal([n_nodes_h2,n_nodes_h3])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_h3]))}
    output_layer={'weights':tf.Variable(tf.random_normal([n_nodes_h3,n_classes])),
                   'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']),hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1) # activation function
    
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2) # activation function
    
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3) # activation function
    

    output = tf.add(tf.matmul(l3,output_layer['weights']),output_layer['biases'])
    
    return output

#Computatinal graph is ready now lets tell tensorflow what to do with it
    
    
    

In [49]:
def train_model(x):
    prediction = model(x) # get output here
    # find: prediction vs known output (i.e., 'y') difference
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)) 
    # reduce the cost using optimizer
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    n_epochs = 5 #10
    
    with tf.Session() as sess:
        #sess.run(tf.initialize_all_variables())
        sess.run(tf.global_variables_initializer())
        for epoch in range (n_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _,c = sess.run([optimizer,cost],feed_dict = {x:epoch_x,y:epoch_y})
                epoch_loss+=c
            print ('Epoch', epoch, 'completed out _of', n_epochs, 'loss:', epoch_loss) 
            
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print ('Accuracy:', accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))
                
    
    

In [50]:
train_model(x)

Epoch 0 completed out _of 5 loss: 226551.395226
Epoch 1 completed out _of 5 loss: 57056.2802792
Epoch 2 completed out _of 5 loss: 35981.9243736
Epoch 3 completed out _of 5 loss: 25686.6214771
Epoch 4 completed out _of 5 loss: 19431.6357813
Accuracy: 0.8858
